In [1]:
from collections import Counter
from math import sqrt
from random import Random
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
import string
import re
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse import csc_matrix
from scipy.sparse import lil_matrix
from scipy.sparse.linalg import svds
#from scipy.stats.stats import spearmanr
from scipy.stats import spearmanr
from random import shuffle
import math
import random
from numpy import *
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
#from sparsesvd import sparsesvd
from numpy.linalg import matrix_rank
from scipy.stats import chi2_contingency
from matplotlib.pyplot import *
from numpy import inf
from scipy.linalg import svd
from foobar import *

In [2]:
#load basic matrix and elements that are neeeded for later
cooccur_dense = np.load("D:\\carme\\bnc\\decomposed matrix\\bnc_cooccur_dense.npy")
vocab = np.load("D:\\carme\\bnc\\decomposed matrix\\bnc_vocab.npy", allow_pickle=True)
vocab = dict(enumerate(vocab.flatten(), 1))[1]

test_files = ['wordsim353.txt', 'men_dataset.txt', 'mturk.txt', 'rarewords.txt', 'simlex999.txt']

dim_range1 = [2, 50]
dim_range2 = list(np.arange(100, 1001, 100))
dim_range3 = list(np.arange(2000, 10001, 1000))
dim_range4 = [11000]
dim_range = [*dim_range1, *dim_range2, *dim_range3, *dim_range4]
dim_range
eig_weight = [0, 0.5, 1, 1.2]
eig_weight_str = ['0', '0.5', '1', '1.2']
bnc_rank = 11332

In [3]:
#To look at word pairs in BNC

cooccur_vectors_words = fittingmatrixvectors(cooccur_dense)

bnc_test_word_pairs = []

for i in range(len(test_files)):
    
    path_input = "D:\\carme\\test dataset\\similarities\\"+test_files[i]
    
    test_word_pairs_set = read_test_word_pairs_set(path_input)
    
    evaluate_word_pairs_create = evaluate_word_pairs(cooccur_vectors_words, test_word_pairs_set, vocab, "cosine")
    
    bnc_test_word_pairs.append((len(test_word_pairs_set), evaluate_word_pairs_create[1], float(evaluate_word_pairs_create[1]) / len(test_word_pairs_set)))
#table 1
bnc_test_word_pairs

[(353, 276, 0.7818696883852692),
 (3000, 1925, 0.6416666666666667),
 (287, 197, 0.686411149825784),
 (2034, 204, 0.10029498525073746),
 (999, 847, 0.8478478478478478)]

In [4]:
#Evaluate TTEST, PMI, and PPMI

ttest = build_ttest(cooccur_dense)
ttest_vectors_words = fittingmatrixvectors(ttest)
ws_score_ttest = []
for i in range(len(test_files)):
    path_input = "D:\\carme\\test dataset\\similarities\\"+test_files[i]
    ws_score_ttest.append(("ttest", test_files[i], ws_evaluate(ttest_vectors_words, vocab, mearsure = 'cosine', path = path_input)))      
    

pmi = build_count_transform(cooccur_dense, contratype = "pmi")
pmi_vectors_words = fittingmatrixvectors(pmi)
ws_score_pmi = []
for i in range(len(test_files)):
    path_input = "D:\\carme\\test dataset\\similarities\\"+test_files[i]
    ws_score_pmi.append(("pmi", test_files[i], ws_evaluate(pmi_vectors_words, vocab, mearsure = 'cosine', path = path_input)))  

ppmi = build_count_transform(cooccur_dense, contratype = "ppmi")
ppmi_vectors_words = fittingmatrixvectors(ppmi)
ws_score_ppmi = []

for i in range(len(test_files)):
    path_input = "D:\\carme\\test dataset\\similarities\\"+test_files[i]
    ws_score_ppmi.append(("ppmi", test_files[i], ws_evaluate(ppmi_vectors_words, vocab, mearsure = 'cosine', path = path_input)))      

C:\Users\qi000005\OneDrive - Universiteit Utrecht\qi000005\paper 3\Correspondence analysis to PMI-based word embeddings\CARME\foobar.py:77: RuntimeWarning: divide by zero encountered in log2
  pmi = np.log2(pmi)
C:\Users\qi000005\OneDrive - Universiteit Utrecht\qi000005\paper 3\Correspondence analysis to PMI-based word embeddings\CARME\foobar.py:87: RuntimeWarning: divide by zero encountered in log2
  pmi = np.log2(pmi)


In [5]:
#Save the evaluation of TTEST, PMI, and PPMI

np.save("D:\\carme\\bnc\\table2\\bnc_ws_score_ttest.npy", ws_score_ttest)  
np.save("D:\\carme\\bnc\\table2\\bnc_ws_score_pmi.npy", ws_score_pmi)  
np.save("D:\\carme\\bnc\\table2\\bnc_ws_score_ppmi.npy", ws_score_ppmi)

In [ ]:
#create the word-context matrix of PMI, PPMI, ROOT, ROOTROOT

decomposedmatrix = ["pmi", "ppmi", "root", "rootroot"]
for i in range(len(decomposedmatrix)):
    temp = build_count_transform(cooccur_dense, contratype = decomposedmatrix[i])
    np.save("D:\\carme\\bnc\\decomposed matrix\\bnc_"+decomposedmatrix[i]+".npy", temp)
#copy bnc_cooccur_dense and chane the name into bnc_raw



In [5]:
#Evaluate ROOT-TTEST and save the evaluation

bnc_root = np.load("D:\\carme\\bnc\\decomposed matrix\\bnc_root.npy")
root_ttest = build_ttest(bnc_root)
root_ttest_vectors_words = fittingmatrixvectors(root_ttest)
ws_score_root_ttest = []
for i in range(len(test_files)):
    path_input = "D:\\carme\\test dataset\\similarities\\"+test_files[i]
    ws_score_root_ttest.append(("root_ttest", test_files[i], ws_evaluate(root_ttest_vectors_words, vocab, mearsure = 'cosine', path = path_input)))      
np.save("D:\\carme\\bnc\\table2\\bnc_ws_score_root_ttest.npy", ws_score_root_ttest)  
ws_score_root_ttest    

[('root_ttest', 'wordsim353.txt', 0.539189142278815),
 ('root_ttest', 'men_dataset.txt', 0.2932324188288258),
 ('root_ttest', 'mturk.txt', 0.6594621929307336),
 ('root_ttest', 'rarewords.txt', 0.47734781813627786),
 ('root_ttest', 'simlex999.txt', 0.27994840678708527)]

In [6]:
#Evaluate ROOTROOT-TTEST and save the evaluation

bnc_rootroot = np.load("D:\\carme\\bnc\\decomposed matrix\\bnc_rootroot.npy")
rootroot_ttest = build_ttest(bnc_rootroot)
rootroot_ttest_vectors_words = fittingmatrixvectors(rootroot_ttest)
ws_score_rootroot_ttest = []
for i in range(len(test_files)):
    path_input = "D:\\carme\\test dataset\\similarities\\"+test_files[i]
    ws_score_rootroot_ttest.append(("rootroot_ttest", test_files[i], ws_evaluate(rootroot_ttest_vectors_words, vocab, mearsure = 'cosine', path = path_input)))      
np.save("D:\\carme\\bnc\\table2\\bnc_ws_score_rootroot_ttest.npy", ws_score_rootroot_ttest)  
ws_score_rootroot_ttest  

[('rootroot_ttest', 'wordsim353.txt', 0.49516590917845443),
 ('rootroot_ttest', 'men_dataset.txt', 0.2630053209840326),
 ('rootroot_ttest', 'mturk.txt', 0.6159739847497702),
 ('rootroot_ttest', 'rarewords.txt', 0.4538918224648214),
 ('rootroot_ttest', 'simlex999.txt', 0.23853544432680374)]

In [ ]:
#create the word-context matrix of STRATOS-TTEST

decomposedmatrix = ["rootcca"]
for i in range(len(decomposedmatrix)):
    temp = build_count_transform(cooccur_dense, contratype = decomposedmatrix[i])
    np.save("D:\\carme\\bnc\\decomposed matrix\\bnc_"+decomposedmatrix[i]+".npy", temp)
#copy bnc_cooccur_dense and chane the name into bnc_raw



In [7]:
#Evaluate the STRATOS-TTEST and save the evaluation

bnc_rootcca = np.load("D:\\carme\\bnc\\decomposed matrix\\bnc_rootcca.npy")
rootcca_vectors_words = fittingmatrixvectors(bnc_rootcca)
ws_score_rootcca = []
for i in range(len(test_files)):
    path_input = "D:\\carme\\test dataset\\similarities\\"+test_files[i]
    ws_score_rootcca.append(("rootcca", test_files[i], ws_evaluate(rootcca_vectors_words, vocab, mearsure = 'cosine', path = path_input)))      
np.save("D:\\carme\\bnc\\table2\\bnc_ws_score_rootcca.npy", ws_score_rootcca)  
ws_score_rootcca    

[('rootcca', 'wordsim353.txt', 0.31354467286412),
 ('rootcca', 'men_dataset.txt', 0.12972319968325272),
 ('rootcca', 'mturk.txt', 0.5252851890867198),
 ('rootcca', 'rarewords.txt', 0.19622644483481302),
 ('rootcca', 'simlex999.txt', 0.1245647519534025)]

In [3]:
#Evaluate the WPMI and save the evaluation


bnc_pmi = np.load("D:\\carme\\bnc\\decomposed matrix\\bnc_pmi.npy")

beta = 1
a = 1
b = 1
P = cooccur_dense / np.sum(cooccur_dense)
sum_w = np.array(P.sum(axis = 1))
sum_c = np.array(P.sum(axis=0)).T
sum_c_beta = (sum_c ** (beta)) / np.sum(sum_c ** (beta)) 
sum_w_pro_root = sum_w ** (a/2)
sum_c_pro_root = sum_c_beta ** (b/2)
        
sum_w_pro_root_diag = np.diag(sum_w_pro_root)
sum_c_pro_root_diag = np.diag(sum_c_pro_root)
  
bnc_pmi_gsvd = sum_w_pro_root_diag @ bnc_pmi @ sum_c_pro_root_diag

pmi_g_vectors_words = fittingmatrixvectors(bnc_pmi_gsvd)
ws_score_pmi_g = []

for i in range(len(test_files)):
    path_input = "D:\\carme\\test dataset\\similarities\\"+test_files[i]
    ws_score_pmi_g.append(("pmi_g", test_files[i], ws_evaluate(pmi_g_vectors_words, vocab, mearsure = 'cosine', path = path_input))) 
    
np.save("D:\\carme\\bnc\\table2\\bnc_ws_score_pmi_g.npy", ws_score_pmi_g)      
ws_score_pmi_g

[('pmi_g', 'wordsim353.txt', 0.22059890392876363),
 ('pmi_g', 'men_dataset.txt', 0.1710600317598314),
 ('pmi_g', 'mturk.txt', 0.41717489588729745),
 ('pmi_g', 'rarewords.txt', 0.25480494207036325),
 ('pmi_g', 'simlex999.txt', 0.11793292751302933)]

In [ ]:
#Create the PMI-SVD, PPMI-SVD, and RAW-CA, ROOT-CA, ROOTROOT-CA
svddecomposedmatrix = ["pmi", "ppmi"]
for i in range(len(svddecomposedmatrix)):
    print(i)
    temp = np.load("D:\\carme\\bnc\\decomposed matrix\\bnc_"+svddecomposedmatrix[i]+".npy")
    svd_u, svd_s, svd_v = svdcadense(temp, method = "svd")
    np.save("D:\\carme\\bnc\\decomposed matrix\\bnc_"+svddecomposedmatrix[i]+"_svd_u.npy", svd_u)
    np.save("D:\\carme\\bnc\\decomposed matrix\\bnc_"+svddecomposedmatrix[i]+"_svd_s.npy", svd_s)
    np.save("D:\\carme\\bnc\\decomposed matrix\\bnc_"+svddecomposedmatrix[i]+"_svd_v.npy", svd_v)
    
cadecomposedmatrix = ["raw", "root", "rootroot"]
for i in range(len(cadecomposedmatrix)):
    print(i)
    temp = np.load("D:\\carme\\bnc\\decomposed matrix\\bnc_"+cadecomposedmatrix[i]+".npy")
    ca_u, ca_s, ca_v = svdcadense(temp, method = "ca")
    np.save("D:\\carme\\bnc\\decomposed matrix\\bnc_"+cadecomposedmatrix[i]+"_ca_u.npy", ca_u)
    np.save("D:\\carme\\bnc\\decomposed matrix\\bnc_"+cadecomposedmatrix[i]+"_ca_s.npy", ca_s)
    np.save("D:\\carme\\bnc\\decomposed matrix\\bnc_"+cadecomposedmatrix[i]+"_ca_v.npy", ca_v)

In [3]:
#Create the ROOT-CCA

svddecomposedmatrix = ["rootcca"]
for i in range(len(svddecomposedmatrix)):
    print(i)
    temp = np.load("D:\\carme\\bnc\\decomposed matrix\\bnc_"+svddecomposedmatrix[i]+".npy")
    svd_u, svd_s, svd_v = svdcadense(temp, method = "svd")
    np.save("D:\\carme\\bnc\\decomposed matrix\\bnc_"+svddecomposedmatrix[i]+"_svd_u.npy", svd_u)
    np.save("D:\\carme\\bnc\\decomposed matrix\\bnc_"+svddecomposedmatrix[i]+"_svd_s.npy", svd_s)
    np.save("D:\\carme\\bnc\\decomposed matrix\\bnc_"+svddecomposedmatrix[i]+"_svd_v.npy", svd_v)

0


In [ ]:
#Evaluate PMI-SVD, PPMI-SVD, and RAW-CA, ROOT-CA, ROOTROOT-CA
svddecomposedmatrix = ["pmi", "ppmi"]

for q in range(len(svddecomposedmatrix)):
    print(q)
    svd_u = np.load("D:\\carme\\bnc\\decomposed matrix\\bnc_"+svddecomposedmatrix[q]+"_svd_u.npy")
    svd_s = np.load("D:\\carme\\bnc\\decomposed matrix\\bnc_"+svddecomposedmatrix[q]+"_svd_s.npy")
    svd_u = np.fliplr(svd_u)
    svd_s = svd_s[::-1]
    svd_ws_score_cos_all = list()
    for k in range(len(test_files)):
        print("k = " + str(test_files[k]))
        test_type = test_files[k]
    
        path_input = "D:\\carme\\test dataset\\similarities\\"+test_files[k]

        svd_ws_score_cos = pd.DataFrame()
    
        for i in range(len(eig_weight)):
            print("i = " + str(eig_weight[i]))
    
            svd_w = svd_u * np.power(svd_s, eig_weight[i])

            svd_score = []

            for j in dim_range:
                vectors_words = svdvectors(svd_w, max_dim = svd_u.shape[1], dim = j)
        
                svd_score_dim = ws_evaluate(vectors_words, vocab, mearsure = 'cosine', path = path_input)
            
                svd_score.append(svd_score_dim)

            svd_ws_score_cos[eig_weight_str[i]] = svd_score
        svd_ws_score_cos_all.append(svd_ws_score_cos)
    
    np.save("D:\\carme\\bnc\\bnc_"+svddecomposedmatrix[q]+"_svd_ws_score_cos_all.npy", svd_ws_score_cos_all)

    
cadecomposedmatrix = ["raw", "root", "rootroot"]

for q in range(len(cadecomposedmatrix)):
    print(q)
    ca_u = np.load("D:\\carme\\bnc\\decomposed matrix\\bnc_"+cadecomposedmatrix[q]+"_ca_u.npy")
    ca_s = np.load("D:\\carme\\bnc\\decomposed matrix\\bnc_"+cadecomposedmatrix[q]+"_ca_s.npy")
    ca_u = np.fliplr(ca_u)
    ca_s = ca_s[::-1]
    ca_ws_score_cos_all = list()
    for k in range(len(test_files)):
        print("k = " + str(test_files[k]))
        test_type = test_files[k]
    
        path_input = "D:\\carme\\test dataset\\similarities\\"+test_files[k]

        ca_ws_score_cos = pd.DataFrame()
    
        for i in range(len(eig_weight)):
            print("i = " + str(eig_weight[i]))
    
            ca_w = ca_u * np.power(ca_s, eig_weight[i])

            ca_score = []

            for j in dim_range:
                vectors_words = svdvectors(ca_w, max_dim = ca_u.shape[1], dim = j)
        
                ca_score_dim = ws_evaluate(vectors_words, vocab, mearsure = 'cosine', path = path_input)
            
                ca_score.append(ca_score_dim)

            ca_ws_score_cos[eig_weight_str[i]] = ca_score
        ca_ws_score_cos_all.append(ca_ws_score_cos)
    
    np.save("D:\\carme\\bnc\\bnc_"+cadecomposedmatrix[q]+"_ca_ws_score_cos_all.npy", ca_ws_score_cos_all)

In [4]:
#create and evaluate ROOT-CCA
svddecomposedmatrix = ["rootcca"]

for q in range(len(svddecomposedmatrix)):
    print(q)
    svd_u = np.load("D:\\carme\\bnc\\decomposed matrix\\bnc_"+svddecomposedmatrix[q]+"_svd_u.npy")
    svd_s = np.load("D:\\carme\\bnc\\decomposed matrix\\bnc_"+svddecomposedmatrix[q]+"_svd_s.npy")
    svd_u = np.fliplr(svd_u)
    svd_s = svd_s[::-1]
    svd_ws_score_cos_all = list()
    for k in range(len(test_files)):
        print("k = " + str(test_files[k]))
        test_type = test_files[k]
    
        path_input = "D:\\carme\\test dataset\\similarities\\"+test_files[k]

        svd_ws_score_cos = pd.DataFrame()
    
        for i in range(len(eig_weight)):
            print("i = " + str(eig_weight[i]))
    
            svd_w = svd_u * np.power(svd_s, eig_weight[i])

            svd_score = []

            for j in dim_range:
                vectors_words = svdvectors(svd_w, max_dim = svd_u.shape[1], dim = j)
        
                svd_score_dim = ws_evaluate(vectors_words, vocab, mearsure = 'cosine', path = path_input)
            
                svd_score.append(svd_score_dim)

            svd_ws_score_cos[eig_weight_str[i]] = svd_score
        svd_ws_score_cos_all.append(svd_ws_score_cos)
    
    np.save("D:\\carme\\bnc\\bnc_"+svddecomposedmatrix[q]+"_svd_ws_score_cos_all.npy", svd_ws_score_cos_all)


0
k = wordsim353.txt
i = 0
i = 0.5
i = 1
i = 1.2
k = men_dataset.txt
i = 0
i = 0.5
i = 1
i = 1.2
k = mturk.txt
i = 0
i = 0.5
i = 1
i = 1.2
k = rarewords.txt
i = 0
i = 0.5
i = 1
i = 1.2
k = simlex999.txt
i = 0
i = 0.5
i = 1
i = 1.2


In [4]:
#Calculate the rank of RAW, PMI, PPMI, ROOT, ROOTROOT
svddecomposedmatrix = ["raw", "pmi", "ppmi", "root", "rootroot"]
for i in range(len(svddecomposedmatrix)):
    print(i)
    temp = np.load("D:\\carme\\bnc\\decomposed matrix\\bnc_"+svddecomposedmatrix[i]+".npy")
    print(matrix_rank(temp))

0
11332
1
11332
2
11332
3
11332
4
11332


In [6]:
#Calculate the rank of STRATOS-TTEST

svddecomposedmatrix = ["rootcca"]
for i in range(len(svddecomposedmatrix)):
    print(i)
    temp = np.load("D:\\carme\\bnc\\decomposed matrix\\bnc_"+svddecomposedmatrix[i]+".npy")
    print(matrix_rank(temp))

0
11332


In [ ]:
#Create and evaluate PMI-GSVD

svddecomposedmatrix = ["pmi"]
for i in range(len(svddecomposedmatrix)):
    print(i)
    temp = np.load("D:\\carme\\bnc\\decomposed matrix\\bnc_"+svddecomposedmatrix[i]+".npy")
    gsvd_u, gsvd_s, gsvd_v = build_gsvd(temp, cooccur_dense, 1, 1)
    np.save("D:\\carme\\bnc\\decomposed matrix\\bnc_"+svddecomposedmatrix[i]+"_gsvd_u.npy", gsvd_u)
    np.save("D:\\carme\\bnc\\decomposed matrix\\bnc_"+svddecomposedmatrix[i]+"_gsvd_s.npy", gsvd_s)
    np.save("D:\\carme\\bnc\\decomposed matrix\\bnc_"+svddecomposedmatrix[i]+"_gsvd_v.npy", gsvd_v)


svddecomposedmatrix = ["pmi"]

for q in range(len(svddecomposedmatrix)):
    print(q)
    svd_u = np.load("D:\\carme\\bnc\\decomposed matrix\\bnc_"+svddecomposedmatrix[q]+"_gsvd_u.npy")
    svd_s = np.load("D:\\carme\\bnc\\decomposed matrix\\bnc_"+svddecomposedmatrix[q]+"_gsvd_s.npy")
    gsvd_ws_score_cos_all = list()
    for k in range(len(test_files)):
        print("k = " + str(test_files[k]))
        test_type = test_files[k]
    
        path_input = "D:\\carme\\test dataset\\similarities\\"+test_files[k]

        svd_ws_score_cos = pd.DataFrame()
    
        for i in range(len(eig_weight)):
            print("i = " + str(eig_weight[i]))
    
            svd_w = svd_u * np.power(svd_s, eig_weight[i])

            svd_score = []

            for j in dim_range:
                vectors_words = svdvectorsincrease(svd_w, dim = j)
        
                svd_score_dim = ws_evaluate(vectors_words, vocab, mearsure = 'cosine', path = path_input)
            
                svd_score.append(svd_score_dim)

            svd_ws_score_cos[eig_weight_str[i]] = svd_score
        gsvd_ws_score_cos_all.append(svd_ws_score_cos)
    
    np.save("D:\\carme\\bnc\\bnc_"+svddecomposedmatrix[q]+"_gsvd_ws_score_cos_all.npy", gsvd_ws_score_cos_all)

    

In [ ]:
#create TTEST, WPMI, ROOT-TTEST, ROOTROOT-TTEST

bnc_ttest = build_ttest(bnc_cooccur_dense)
np.save("D:\\carme\\bnc\\decomposed matrix\\bnc_ttest.npy", bnc_ttest)

bnc_pmi = np.load("D:\\carme\\bnc\\decomposed matrix\\bnc_pmi.npy")

#bnc_ppmi = np.where(bnc_pmi < 0, 0, bnc_pmi)

beta = 1
a = 1
b = 1
P = bnc_cooccur_dense / np.sum(bnc_cooccur_dense)
sum_w = np.array(P.sum(axis = 1))
sum_c = np.array(P.sum(axis=0)).T
sum_c_beta = (sum_c ** (beta)) / np.sum(sum_c ** (beta)) 
sum_w_pro_root = sum_w ** (a/2)
sum_c_pro_root = sum_c_beta ** (b/2)
        
sum_w_pro_root_diag = np.diag(sum_w_pro_root)
sum_c_pro_root_diag = np.diag(sum_c_pro_root)
  
bnc_pmi_gsvd = sum_w_pro_root_diag @ bnc_pmi @ sum_c_pro_root_diag
np.save("D:\\carme\\bnc\\decomposed matrix\\bnc_pmi_gsvd.npy", bnc_pmi_gsvd)



bnc_root = np.load("D:\\carme\\bnc\\decomposed matrix\\bnc_root.npy")
bnc_root_ttest = build_ttest(bnc_root)
np.save("D:\\carme\\bnc\\decomposed matrix\\bnc_root_ttest.npy", bnc_root_ttest)


bnc_rootroot = np.load("D:\\carme\\bnc\\decomposed matrix\\bnc_rootroot.npy")
bnc_rootroot_ttest = build_ttest(bnc_rootroot)
np.save("D:\\carme\\bnc\\decomposed matrix\\bnc_rootroot_ttest.npy", bnc_rootroot_ttest)

#bnc_rootcca = np.load("D:\\carme\\bnc\\decomposed matrix\\bnc_rootcca.npy")